<center><h1> Tangelo Fintech </h1></center>
<center><h3> Python Backend Technical Test </h3></center>
<center><h3> Landneyker Betancourth </h3></center>

## Setup:

In [1]:
import requests
import time
import hashlib
import pandas as pd
import json

## Getting data:

In [2]:
url='https://restcountries.com/v3.1/all'  # This is a link of rest country API
data_url = requests.get(url)              # Request data to URL
data = data_url.json()                    # JSON data for display

print(data[0])                            # Displaying data content
print(data[0].keys())                     # Duisplaying keys for data content

{'name': {'common': 'DR Congo', 'official': 'Democratic Republic of the Congo', 'nativeName': {'fra': {'official': 'République démocratique du Congo', 'common': 'RD Congo'}, 'kon': {'official': 'Repubilika ya Kongo Demokratiki', 'common': 'Repubilika ya Kongo Demokratiki'}, 'lin': {'official': 'Republiki ya Kongó Demokratiki', 'common': 'Republiki ya Kongó Demokratiki'}, 'lua': {'official': 'Ditunga dia Kongu wa Mungalaata', 'common': 'Ditunga dia Kongu wa Mungalaata'}, 'swa': {'official': 'Jamhuri ya Kidemokrasia ya Kongo', 'common': 'Jamhuri ya Kidemokrasia ya Kongo'}}}, 'tld': ['.cd'], 'cca2': 'CD', 'ccn3': '180', 'cca3': 'COD', 'cioc': 'COD', 'independent': True, 'status': 'officially-assigned', 'unMember': True, 'currencies': {'CDF': {'name': 'Congolese franc', 'symbol': 'FC'}}, 'idd': {'root': '+2', 'suffixes': ['43']}, 'capital': ['Kinshasa'], 'altSpellings': ['CD', 'DR Congo', 'Congo-Kinshasa', 'Congo, the Democratic Republic of the', 'DRC'], 'region': 'Africa', 'subregion': 'M

## Making dataframe:

In [3]:
# Making DataFrame
dict_data = {}              # Dictionary to save data
list_region = []            # List to add the data of the region
list_city_name = []         # List to add the data of the city name
list_language = []          # List to add the data of the language
list_time = []              # List to add the data of the time it takes to create a row like a string for visualization
list_time_num = []          # List to add the data of the time it takes to create a row like a float for getting max, min and average

for i in range(len(data)):  
  start_time = time.time()  # Time to calculate the creation of a row 
  
  if data[i].get('languages') == None:              # Key validation
    lang = 'Not Found'                              # Add default value
  else:
    lang = list(data[i]['languages'].values())[0]   # If key exist add real value
  
  time_num = (time.time() - start_time) * 1000      # Time convertion to miliseconds (ms)

  list_region.append(data[i]['region'])                                # Add region to list
  list_city_name.append(data[i]['name']['common'])                     # Add common name to list
  list_language.append(hashlib.sha1(lang.encode("utf-8")).hexdigest()) # Encrypting data and using hexdigest() to display 
  list_time_num.append(time_num)                                       # Add time to list -> float for calculate max, min and average
  list_time.append(str(time_num)[:6] + ' ms')                          # Add time to list -> str to add 'ms'

# Add data to dict
dict_data['Region'] = list_region
dict_data['City Name'] = list_city_name
dict_data['Language'] = list_language
dict_data['Time'] = list_time

df = pd.DataFrame(dict_data)    # Making DataFrame

## Displaying dataframe:

In [4]:
df

,Region,City Name,Language,Time
0,Africa,DR Congo,44389f6a466eaabf5c1db729e369511134e2b03b,0.0054 ms
1,Europe,North Macedonia,b3fe571f349f4b42daacd19bf3d1a708f4bec7cd,0.0088 ms
2,Oceania,Tonga,649df08a448ee3fa90f3746baaf6b0907df42c91,0.0021 ms
3,Europe,Poland,d011986407b1110e57283981ea38a369075023b8,0.0026 ms
4,Africa,Angola,23882c575954a0789bf02aba9e6dd01f539bc738,0.0023 ms
...,...,...,...,...
245,Africa,Libya,af4f4762f9bd3f0f4a10caf5b6e63dc4ce543724,0.0023 ms
246,Asia,Saudi Arabia,af4f4762f9bd3f0f4a10caf5b6e63dc4ce543724,0.0026 ms
247,Asia,Malaysia,649df08a448ee3fa90f3746baaf6b0907df42c91,0.0023 ms
248,Americas,Jamaica,649df08a448ee3fa90f3746baaf6b0907df42c91,0.0026 ms


## Processing time:

In [5]:
col_time = df['Time']

Max time:

In [6]:
col_time.max()

'0.1063 ms'

Min time:

In [7]:
col_time.min()

'0.0011 ms'

Average time:

In [8]:
#col_time.mean()
str(sum(list_time_num) / len(list_time_num))[:6]

'0.0028'

## Making and download JSON file from data dict:

In [9]:
# Making and download JSON file from data dict
data_json = json.dumps(dict_data)     # Dict to JSON
json_file = open('data.json', 'w')    # Creating the file
json_file.write(data_json)            # Writing data
json_file.close()                     # Saving data JSON as data.json
print(data_json)                      # Displaying data.json

{"Region": ["Africa", "Europe", "Oceania", "Europe", "Africa", "Africa", "Africa", "Africa", "Africa", "Europe", "Africa", "Europe", "Africa", "Europe", "Asia", "Americas", "Americas", "Americas", "Asia", "Oceania", "Americas", "Antarctic", "Africa", "Africa", "Asia", "Europe", "Europe", "Africa", "Europe", "Africa", "Europe", "Americas", "Africa", "Americas", "Europe", "Americas", "Europe", "Europe", "Africa", "Europe", "Africa", "Africa", "Europe", "Europe", "Europe", "Africa", "Africa", "Africa", "Asia", "Africa", "Americas", "Americas", "Americas", "Americas", "Americas", "Americas", "Europe", "Americas", "Europe", "Europe", "Africa", "Asia", "Americas", "Asia", "Oceania", "Africa", "Asia", "Europe", "Africa", "Asia", "Asia", "Asia", "Oceania", "Americas", "Europe", "Europe", "Africa", "Americas", "Americas", "Americas", "Europe", "Oceania", "Europe", "Americas", "Oceania", "Americas", "Asia", "Africa", "Asia", "Americas", "Africa", "Europe", "Africa", "Oceania", "Americas", "Ameri

## Storing the data in the database:

In [23]:
# Making database connection
import sqlite3
from sqlite3 import Error

def sql_conn():
  try:
    conn = sqlite3.connect('contries_db.db')     # Creating DB
    return conn
  except Error:
    print(Error)

def sql_table(conn, table, data):
  cursor_obj = conn.cursor()                     # Cursor object to execute statements
  cursor_obj.execute('CREATE TABLE IF NOT EXISTS ' + table + '(' + data + ')')
  conn.commit()

def sql_insert(conn, table, data, df):
  cursor_obj = conn.cursor()                     # Cursor object to execute statements
  for row in df.itertuples():                    # Insert dataframe rows one by one
    sql_sentence = f"INSERT INTO " + table + f"(" + data + f") VALUES({row[0]}, '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}')"
    cursor_obj.execute(sql_sentence)
  conn.commit()
  print('We have inserted', row[0]+1, 'records to the table.')

In [24]:
# Making the connection
conn = sql_conn()
cursor_obj = conn.cursor()

In [25]:
# Creating table and columns
countries_table = 'countries'
data_table = 'id integer PRIMARY KEY, region text, city_name text, language text, time text'
data_table_names = 'id, region, city_name, language, time'

sql_table(conn, countries_table, data_table)

In [26]:
# Insert data into the table
sql_insert(conn, countries_table, data_table_names, df)

We have inserted 250 records to the table.
